In [3]:
from cub2011 import *
from torchvision.models import alexnet, AlexNet_Weights
from torchvision import transforms
import torch.nn as nn
import torch

# Define the transform with augmentation
transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # 随机裁剪并调整大小
    transforms.RandomHorizontalFlip(),                    # 随机水平翻转
    # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # 随机颜色抖动
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Define the transform
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
# ])

# load dataset
train = Cub2011("", transform=transform)
test = Cub2011("", train=False, transform=transform)

# see example data
# img, target = train.__getitem__(999)
# img.show()

# load alexnet
# net = alexnet(weights=AlexNet_Weights.IMAGENET1K_V1)
net = alexnet(weights=None)
net.classifier[-1] = nn.Linear(4096, 200)

# move the net to GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(device)
net.to(device)


Files already downloaded and verified
Files already downloaded and verified


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [4]:
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
from torch.utils.tensorboard import SummaryWriter


# Loss
criterion = nn.CrossEntropyLoss()

# Separate parameters into two groups
base_lr = 0.1
finetune_rate = 1
finetune_lr = finetune_rate * base_lr
params_to_update = [
    {'params': net.features.parameters(), 'lr': finetune_lr},  # Base layers
    {'params': net.classifier[:-1].parameters(), 'lr': finetune_lr},  # Classifier layers except the last one
    {'params': net.classifier[-1].parameters(), 'lr': base_lr}  # The last layer
]


# optimizer
optimizer = optim.SGD(params_to_update, momentum=0.9, weight_decay=1e-4)

# Learning rate sechuler
scheduler = StepLR(optimizer, step_size=50, gamma=0.1)

train_loader = DataLoader(train, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test, batch_size=32, shuffle=False, num_workers=4)

# TENSORBOARD
writer = SummaryWriter(log_dir='Alexnet/alexnet-lre-1-randomized')

# training loop
num_epochs = 110
for epoch in range(num_epochs):
    # train
    net.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss:.3f}", end=", ")
    
    # Write training loss to TensorBoard
    writer.add_scalar('Training Loss', train_loss, epoch)

    # validation
    net.eval()
    correct = 0
    total = 0
    running_loss = 0.0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move data to the device
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_loss = running_loss / len(test_loader)
    accuracy = 100 * correct / total
    print(f'Validation Loss: {val_loss:.3f}', end=", ")
    print(f'Accuracy on test set: {accuracy:.3f}%')

    # Write validation loss and accuracy to TensorBoard
    writer.add_scalar('Validation Loss', val_loss, epoch)
    writer.add_scalar('Accuracy', accuracy, epoch)

    # Update the learning rate
    scheduler.step()
    
# Close the TensorBoard writer
writer.close()


Epoch 1/110, Training Loss: 5.312, Validation Loss: 5.299, Accuracy on test set: 0.501%
Epoch 2/110, Training Loss: 5.311, Validation Loss: 5.299, Accuracy on test set: 0.518%
Epoch 3/110, Training Loss: 5.311, Validation Loss: 5.299, Accuracy on test set: 0.518%
Epoch 4/110, Training Loss: 5.311, Validation Loss: 5.299, Accuracy on test set: 0.518%
Epoch 5/110, Training Loss: 5.311, Validation Loss: 5.299, Accuracy on test set: 0.518%
Epoch 6/110, Training Loss: 5.311, Validation Loss: 5.299, Accuracy on test set: 0.518%
Epoch 7/110, Training Loss: 5.311, Validation Loss: 5.298, Accuracy on test set: 0.518%
Epoch 8/110, Training Loss: 5.311, Validation Loss: 5.299, Accuracy on test set: 0.518%
Epoch 9/110, Training Loss: 5.311, Validation Loss: 5.299, Accuracy on test set: 0.518%
Epoch 10/110, Training Loss: 5.311, Validation Loss: 5.298, Accuracy on test set: 0.518%
Epoch 11/110, Training Loss: 5.311, Validation Loss: 5.299, Accuracy on test set: 0.501%
Epoch 12/110, Training Loss: 5

In [5]:
# !tensorboard --logdir=runs